In [3]:
%pip install setuptools
%pip install gdown
%pip install langchain
%pip install opensearch-py
%pip install openai
%pip install openai[embeddings]
%pip install -U sentence-transformers
%pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 5.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.4/117.4 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.8/282.8 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 6.9 M

In [1]:
from langchain.embeddings import HuggingFaceEmbeddings
import warnings
import uuid
from uuid import uuid4
from langchain.vectorstores import OpenSearchVectorSearch
from urllib3.exceptions import InsecureRequestWarning
import json
import gdown
from opensearchpy import OpenSearch
from langchain.docstore.document import Document
from sentence_transformers import SentenceTransformer

/home/tahaAmineMiri/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)

In [3]:
# Suppress only the single warning from urllib3
warnings.filterwarnings(action='ignore', category=InsecureRequestWarning)
warnings.filterwarnings(action='ignore', category=UserWarning)

In [4]:
documents = []

print(len(documents))

with open('output.json', 'r') as f:
    documents = json.load(f)

print(len(documents))

print(documents[0])

0
86312
{'metadata': {'url': 'https://insea.ac.ma', 'title': 'INSEA', 'source_type': 'webpage'}, 'content': "Institut National de Statistique et d'Economie Appliquée  Le lundi 06 septembre 2021 à 10 heures, il sera procédé, dans la salle de réunions n° 01 de l’Institut National de Statistique et d’Economie"}


In [5]:
# Generate UUIDs for documents
for doc in documents:
  doc["id"] = str(uuid.uuid4()) 

In [6]:
# Convert documents to LangChain Document format  
dict_documents = []
for d in documents:
    doc = Document(page_content=d["content"],  metadata = d["metadata"], id = d["id"])
    dict_documents.append(doc)

In [7]:
host = 'localhost'
port = 9200
auth = ('admin', 'admin')

client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = False
)

In [8]:
# Create index name
index_name = 'insea_chatbot'

In [ ]:
# Create vector store
vectorstore = OpenSearchVectorSearch.from_documents(
    documents=dict_documents, 
    embedding=embeddings, 
    index_name=index_name
)